In [1]:
from PIL import Image
import h5py
import anubisPlotUtils as anPlot
import json
import numpy as np
import os
import hist as hi
import matplotlib
import matplotlib.pyplot as plt
from itertools import combinations
import matplotlib.colors as colors
matplotlib.use('TkAgg')  # or 'Qt5Agg', 'GTK3Agg', etc.
import mplhep as hep
hep.style.use([hep.style.ATLAS])
import sys
import AnalysisToolAnubis as AT
import seaborn as sns
import pandas as pd
import tkinter as tk
from tkinter import ttk
from pandasgui import show
from matplotlib.patches import Circle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import convolve2d
from scipy.ndimage import label, find_objects
from scipy.spatial.distance import pdist, squareform
from matplotlib.backends.backend_pdf import PdfPages
import ipywidgets as widgets
from IPython.display import display
import sys
from scipy.stats import gaussian_kde
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.ticker import MaxNLocator
import AnubisSuperScript as ass
from functools import partial
import importlib
from tqdm import tqdm
importlib.reload(ass)
import ast

ERROR:root:Cannot activate multiple GUI eventloops


In [2]:

mapping = {
    0: {
        (0, 31): ('rpc0', 'eta'),
        (32, 63): ('rpc0', 'phi1'),
        (64, 95): ('rpc0', 'phi2'),
        (96, 127): ('rpc1', 'eta'),
    },
    1: {
        (0, 31): ('rpc1', 'phi1'),
        (32, 63): ('rpc1', 'phi2'),
        (64, 95): ('rpc2', 'eta'),
        (96, 127): ('rpc2', 'phi1'),
    },
    2: {
        (0, 31): ('rpc2', 'phi2'),
        (32, 63): ('rpc3', 'eta'),
        (64, 95): ('rpc3', 'phi1'),
        (96, 127): ('rpc3', 'phi2'),
    },
    3: {
        (0, 31): ('rpc4', 'eta'),
        (32, 63): ('rpc4', 'phi1'),
        (64, 95): ('rpc4', 'phi2'),
        (96, 127): ('rpc5', 'eta'),
    },
    4: {
        (0, 31): ('rpc5', 'phi1'),
        (32, 63): ('rpc5', 'phi2'),
    },
}

strips_on_steroid = {
    'rpc0': {'phi': [0]},
    'rpc1': {'phi': [0]},
    'rpc2': {'phi': [0]},
    'rpc3': {'phi': [0]},
    'rpc4': {'eta': [31,0], 'phi': [0]},
    'rpc5': {'eta': [31,0], 'phi': [0]},
}


In [3]:
thisData = AT.importDatafile('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubisData\\proAnubis_240403_0227.h5')

In [4]:
thisData = [sublist[:94230] for sublist in thisData]

In [5]:
etaHits, phiHits = ass.divideHitCountsByRPC_Timed(thisData)

In [6]:
df = ass.remake_data(thisData, mapping)

In [55]:
show(df.groupby('event number').head(100))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [7]:
df_clean = df[df['time'] <= 500]

In [8]:
cluster_df = ass.calculate_cluster_metrics_better(df_clean, 5)

In [56]:
show(cluster_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


5ns window 783898

In [10]:
remove_noisy_strips_with_map = partial(ass.remove_noisy_strips, noisy_strips=strips_on_steroid)
anti_steriod_df = cluster_df[cluster_df.apply(lambda row: remove_noisy_strips_with_map(row), axis=1)]
fully_massaged_df = anti_steriod_df[anti_steriod_df['size'] <= 5]

In [96]:
show(fully_massaged_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [9]:
def calculate_metric_for_combo(combo, rpc_heights):
    locations = np.array([c['location'] for c in combo])
    heights = np.array([rpc_heights[c['rpc']] for c in combo])
    uncertainties = np.array([c['uncertainty'] for c in combo])  
    weights = 1 / uncertainties**2

    try:
        coeffs= np.polyfit(heights, locations, 1, cov=False, w=weights)
    except np.linalg.LinAlgError:
        coeffs = [np.inf, np.inf]
        cov = np.array([[np.inf, np.inf], [np.inf, np.inf]])

    slope, intercept = coeffs
    # slope_error, intercept_error = np.sqrt(np.diag(cov))
    slope_error, intercept_error = 0, 0
    predicted = slope * heights + intercept
    residuals = locations - predicted
    RSS = np.sum(residuals ** 2)

    return slope, intercept, slope_error, intercept_error, RSS, combo

def analyze_inter_rpc_hit_with_timing_adjusted(df):
    rpc_time_offsets = {
    ('rpc0', 'eta'): (7.94, 12.48),
    ('rpc0', 'phi'): (-2.38, 13.69),
    ('rpc1', 'eta'): (8.36, 12.22),
    ('rpc1', 'phi'): (-3.79, 13.25),
    ('rpc2', 'eta'): (8.84, 12.56),
    ('rpc2', 'phi'): (-4.35, 13.57),
    ('rpc3', 'eta'): (6.86, 12.41),
    ('rpc3', 'phi'): (-4.3, 13.96),
    ('rpc4', 'eta'): (2.7, 12.37),
    ('rpc4', 'phi'): (-7.89, 13.41),
    ('rpc5', 'eta'): (2.82, 13.05),
    ('rpc5', 'phi'): (9.15, 14.14),
}
    paths = []

    adjusted_muon_speed_cm_ns = 28

    rpc_heights = {
        'rpc0': 0, 
        'rpc1': 0.5, 
        'rpc2': 1.0, 
        'rpc3': 61.5, 
        'rpc4': 121.5, 
        'rpc5': 122.0
    }

    for event_number, event_group in tqdm(df.groupby('event_number'), desc="Processing Events"):
        for direction in ['eta', 'phi']:
            direction_group = event_group[event_group['strip_direction'] == direction]
            all_clusters = []

            unique_rpcs = direction_group['rpc_number'].unique()
            for rpc in unique_rpcs:
                rpc_group = direction_group[direction_group['rpc_number'] == rpc]
                for _, row in rpc_group.iterrows():
                    location_scaling = 3.09375 if direction == 'eta' else 2.8125
                    strip_locations = np.array(row['locations'])
                    non_zero_locations = strip_locations[strip_locations != 0]
                    if non_zero_locations.size > 0:
                        strip_location = non_zero_locations[0] 
                    else:
                        continue 

                    location = strip_location * location_scaling
                    event_time = np.mean(row['times']) - rpc_time_offsets[(rpc, direction)][0]
                    cluster_size_scaled = max(row['size'] * location_scaling, location_scaling) / 2
                    all_clusters.append({
                        'rpc': rpc,
                        'location': location,
                        'event_time': event_time,
                        'uncertainty': cluster_size_scaled,
                        'original_location': strip_location
                    })

            combination_metrics = []
            valid_combinations = [] 
            # for n in range(3, 4): 
            for combo in combinations(all_clusters, 3):
                used_rpcs_set = {c['rpc'] for c in combo}
                
                if not used_rpcs_set.issubset({'rpc0', 'rpc1', 'rpc2'}):
                    metric = calculate_metric_for_combo(combo, rpc_heights)
        
                    # Check RSS threshold here and ensure RPC combination uniqueness
                    if metric[4] <= 10 and len(used_rpcs_set) == len(combo):
                        combination_metrics.append(metric)
                        
            # Move filtering logic outside the loop so it's not reset each time
            for combo_metric in combination_metrics:
                combo = combo_metric[-1]
                if len({c['rpc'] for c in combo}) < len(combo):
                    continue

                time_diffs_are_valid = True
                for i in range(len(combo)):
                    for j in range(i + 1, len(combo)):
                        error_window = rpc_time_offsets[(combo[i]['rpc'], direction)][1] + rpc_time_offsets[(combo[j]['rpc'], direction)][1]
                        
                        # Use direct height differences
                        height_diff = abs(rpc_heights[combo[i]['rpc']] - rpc_heights[combo[j]['rpc']])
                        
                        time_diff = abs(combo[i]['event_time'] - combo[j]['event_time'])
                        expected_time_diff = height_diff / adjusted_muon_speed_cm_ns

                        uncertainty_margin = 5
                        # Use the expected time difference with the error window and uncertainty margin for validation
                        if not (time_diff <= expected_time_diff + error_window + uncertainty_margin):
                            time_diffs_are_valid = False
                            break
                    if not time_diffs_are_valid:
                        break

                if time_diffs_are_valid:
                    valid_combinations.append(combo_metric)

            for valid_combination in valid_combinations:
                paths.append({
                    'Event Number': event_number,
                    'Direction': direction,
                    'Slope': valid_combination[0],
                    'Intercept': valid_combination[1],
                    'Slope_error': valid_combination[2],
                    'Intercept_error': valid_combination[3],
                    'Used Coordinates': [(c['rpc'], c['original_location'], c['event_time']) for c in valid_combination[-1]],
                    'RSS': valid_combination[4],
                })

    path_df = pd.DataFrame(paths)
    return path_df

In [168]:
Reconstruct2 = analyze_inter_rpc_hit_with_timing_adjusted(fully_massaged_df)

Processing Events:   0%|          | 11/94217 [00:00<14:17, 109.83it/s]C:\Users\Peter\AppData\Local\Temp\ipykernel_32180\1308193378.py:8: RankWarning:

Polyfit may be poorly conditioned

Processing Events: 100%|██████████| 94217/94217 [05:34<00:00, 281.31it/s]


In [156]:
show(Reconstruct2)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [139]:
show(fully_massaged_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [144]:
df_etaaaa = fully_massaged_df[fully_massaged_df['strip_direction'] == 'eta']

df_etaaaa['mean_location'] = df_etaaaa['locations'].apply(lambda x: np.mean(x, axis=0))

df_etaaaa['mean_location_str'] = df_etaaaa['mean_location'].apply(lambda x: str(x))

resultaaa = df_etaaaa.groupby(['event_number', 'mean_location_str']).size().reset_index(name='counts')

# Filtering events where the same mean location appeared more than 3 times
filtered_resultaaa = resultaaa[resultaaa['counts'] > 3]

# Outputting the final result
print(filtered_resultaaa[['event_number', 'mean_location_str']])

C:\Users\Peter\AppData\Local\Temp\ipykernel_32180\3657440907.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



        event_number mean_location_str
1193             444       [  0. -17.]
2143             774       [  0. -24.]
3188            1131       [  0. -31.]
3819            1356       [  0. -24.]
4085            1450         [ 0. -2.]
...              ...               ...
263764         91632       [  0. -31.]
264592         91918       [  0. -23.]
265020         92058       [  0. -25.]
265425         92187       [  0. -21.]
265488         92213       [  0. -22.]

[452 rows x 2 columns]


C:\Users\Peter\AppData\Local\Temp\ipykernel_32180\3657440907.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [169]:
file_path = 'ReconNewWeighing.xlsx'

Reconstruct2.to_excel(file_path, index=False, engine='openpyxl')


In [93]:
Filtered_reconstruct = Reconstruct[Reconstruct['RSS'] < 10]

In [18]:
print(np.degrees(np.arctan(1/5)) - 45)

-33.690067525979785


In [49]:
file_path = 'C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\Final Version\\ReconOldWeighing.xlsx'

df_Old = pd.read_excel(file_path, sheet_name=0)

file_path2 = 'C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\Final Version\\ReconNewWeighing.xlsx'

df_New = pd.read_excel(file_path2, sheet_name=0)

In [8]:
show(df_New.head(100))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [53]:
Filtered_df_Old = df_Old[df_Old['RSS'] < 10]
Filtered_df_New = df_New[df_New['RSS'] < 10]

In [51]:

print(Filtered_df_New.head())

print(Filtered_df_New.info())

   Event Number Direction     Slope   Intercept  Slope_error  Intercept_error  \
0             1       phi  0.138889  120.867008            0                0   
1             3       eta  0.353554  -31.025528            0                0   
2             3       eta  0.358126  -31.307596            0                0   
3             3       eta -0.051111   -6.137904            0                0   
4             3       phi -2.112209  143.963397            0                0   

                                    Used Coordinates       RSS  
0  [('rpc1', 43, 241.79), ('rpc3', 46, 238.8), ('...  0.001136  
1  [('rpc0', -10, 249.06), ('rpc1', -10, 249.64),...  0.015626  
2  [('rpc1', -10, 249.64), ('rpc2', -11, 250.16),...  9.533868  
3  [('rpc1', -2, 251.64), ('rpc2', -2, 251.16), (...  0.000580  
4  [('rpc0', 51, 250.04666666666665), ('rpc1', 51...  0.557688  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 334027 entries, 0 to 334026
Data columns (total 8 columns):
 #   Column  

In [23]:
def process_ties(sub_df):
    if len(sub_df) > 1:
        avg_slope = sub_df['Slope'].mean()
        avg_intercept = sub_df['Intercept'].mean()
        sub_df = sub_df.head(1) 
        sub_df['Slope'] = avg_slope 
        sub_df['Intercept'] = avg_intercept
    return sub_df

Filtered_df_Old = (
    df_Old.sort_values(by=['Event Number', 'Direction', 'RSS'])
      .groupby(['Event Number', 'Direction'], as_index=False)
      .apply(lambda x: process_ties(x[x['RSS'] == x['RSS'].min()]))
      .reset_index(drop=True)
)
Filtered_df_New = (
    df_New.sort_values(by=['Event Number', 'Direction', 'RSS'])
      .groupby(['Event Number', 'Direction'], as_index=False)
      .apply(lambda x: process_ties(x[x['RSS'] == x['RSS'].min()]))
      .reset_index(drop=True)
)


In [ ]:
Filtered_df_Old = Filtered_df_Old[Filtered_df_Old['RSS'] < 3]
Filtered_df_New = Filtered_df_New[Filtered_df_New['RSS'] < 3]

In [48]:
show(Filtered_df_Old)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [179]:
eta_df = Filtered_df_New[Filtered_df_New['Direction'] == 'phi']
angles_y_axis = np.degrees(np.arctan(-eta_df['Slope']))

plt.figure(figsize=(20, 12))
plt.hist(angles_y_axis, bins=100, color='skyblue', edgecolor='black')
plt.xlabel('angles (degrees)')
plt.ylabel('Occurance')
plt.title('Cross_Chamber_Angular_Distribution_eta')
plt.grid(axis='y', alpha=0.75)
plt.show()

In [145]:
phi_df = Filtered_df_Old[Filtered_df_Old['Direction'] == 'phi']
angles = np.degrees(np.arctan(phi_df['Slope']))

plt.figure(figsize=(20, 12))
plt.hist(angles, bins=100, color='skyblue', edgecolor='black')
plt.xlabel('angles (degrees)')
plt.ylabel('Occurance')
plt.title('Cross_Chamber_Angular_Distribution_phi')
plt.grid(axis='y', alpha=0.75)
plt.show()

In [97]:
has_eta = Filtered_reconstruct[Filtered_reconstruct['Direction'] == 'eta']['Event Number'].unique()
has_phi = Filtered_reconstruct[Filtered_reconstruct['Direction'] == 'phi']['Event Number'].unique()
events_with_both = np.intersect1d(has_eta, has_phi)

filtered_df = Filtered_reconstruct[Filtered_reconstruct['Event Number'].isin(events_with_both)]

eta_df = filtered_df[filtered_df['Direction'] == 'eta']
angles_y_axis = np.degrees(np.arctan(-eta_df['Slope']))
weights = 1 / eta_df.groupby('Event Number')['Event Number'].transform('count')

# Plotting
plt.figure(figsize=(20, 12))
plt.hist(angles_y_axis, bins=100, weights=weights, color='skyblue', edgecolor='black')
plt.xlabel('Angles (degrees)')
plt.ylabel('Weighted Occurance')
plt.title('Cross_Chamber_Angular_Distribution_eta (weighted)')
plt.grid(axis='y', alpha=0.75)
plt.show()

In [19]:
has_eta = Filtered_reconstruct[Filtered_reconstruct['Direction'] == 'eta']['Event Number'].unique()
has_phi = Filtered_reconstruct[Filtered_reconstruct['Direction'] == 'phi']['Event Number'].unique()
events_with_both = np.intersect1d(has_eta, has_phi)

filtered_df = Filtered_reconstruct[Filtered_reconstruct['Event Number'].isin(events_with_both)]

eta_df = filtered_df[filtered_df['Direction'] == 'phi']
angles_y_axis = np.degrees(np.arctan(eta_df['Slope']))
weights = 1 / eta_df.groupby('Event Number')['Event Number'].transform('count')

# Plotting
plt.figure(figsize=(20, 12))
plt.hist(angles_y_axis, bins=100, weights=weights, color='skyblue', edgecolor='black')
plt.xlabel('Angles (degrees)')
plt.ylabel('Weighted Occurance')
plt.title('Cross_Chamber_Angular_Distribution_phi (weighted)')
plt.grid(axis='y', alpha=0.75)
plt.show()

NameError: name 'Filtered_reconstruct' is not defined

In [186]:
has_eta1 = Filtered_df_Old[Filtered_df_Old['Direction'] == 'eta']['Event Number'].unique()
has_phi1 = Filtered_df_Old[Filtered_df_Old['Direction'] == 'phi']['Event Number'].unique()
events_with_both1 = np.intersect1d(has_eta1, has_phi1)

filtered_df1 = Filtered_df_Old[Filtered_df_Old['Event Number'].isin(events_with_both1)]

eta_df1 = filtered_df1[filtered_df1['Direction'] == 'eta']
phi_df1 = filtered_df1[filtered_df1['Direction'] == 'phi']
angles_y_axis1 = np.degrees(np.arctan(-eta_df1['Slope']))
weights1 = 1 / eta_df1.groupby('Event Number')['Event Number'].transform('count')

has_eta2 = Filtered_df_New[Filtered_df_New['Direction'] == 'eta']['Event Number'].unique()
has_phi2 = Filtered_df_New[Filtered_df_New['Direction'] == 'phi']['Event Number'].unique()
events_with_both2 = np.intersect1d(has_eta2, has_phi2)

filtered_df2 = Filtered_df_New[Filtered_df_New['Event Number'].isin(events_with_both2)]

eta_df2 = filtered_df2[filtered_df2['Direction'] == 'eta']
phi_df2 = filtered_df2[filtered_df2['Direction'] == 'phi']
angles_y_axis2 = np.degrees(np.arctan(-eta_df2['Slope']))
weights2 = 1 / eta_df2.groupby('Event Number')['Event Number'].transform('count')

# Plotting
plt.figure(figsize=(20, 12))
plt.hist(angles_y_axis2, bins=1000, weights=weights2, color='pink', edgecolor='green', alpha = 1, label = 'Atlas On')
plt.hist(angles_y_axis1, bins=1000, weights=weights1, color='skyblue', edgecolor='black', alpha = 0.5, label = 'Atlas Off')
plt.xlabel('Angles (degrees)')
plt.ylabel('Weighted Occurance')
plt.title('Cross_Chamber_Angular_Distribution_eta (weighted)')
plt.grid(axis='y', alpha=0.75)
plt.legend()
plt.show()

In [174]:
has_eta1 = Filtered_df_Old[Filtered_df_Old['Direction'] == 'eta']['Event Number'].unique()
has_phi1 = Filtered_df_Old[Filtered_df_Old['Direction'] == 'phi']['Event Number'].unique()
events_with_both1 = np.intersect1d(has_eta1, has_phi1)

filtered_df1 = Filtered_df_Old[Filtered_df_Old['Event Number'].isin(events_with_both1)]

eta_df1 = Filtered_df_Old[Filtered_df_Old['Direction'] == 'eta']
phi_df1 = Filtered_df_Old[Filtered_df_Old['Direction'] == 'phi']
angles_y_axis1 = np.degrees(np.arctan(-eta_df1['Slope']))
angles_y_axis11 = np.degrees(np.arctan(phi_df1['Slope']))
weights1 = 1 / eta_df1.groupby('Event Number')['Event Number'].transform('count')
weights11 = 1 / phi_df1.groupby('Event Number')['Event Number'].transform('count')

has_eta2 = Filtered_df_New[Filtered_df_New['Direction'] == 'eta']['Event Number'].unique()
has_phi2 = Filtered_df_New[Filtered_df_New['Direction'] == 'phi']['Event Number'].unique()
events_with_both2 = np.intersect1d(has_eta2, has_phi2)

filtered_df2 = Filtered_df_New[Filtered_df_New['Event Number'].isin(events_with_both2)]

eta_df2 = Filtered_df_New[Filtered_df_New['Direction'] == 'eta']
phi_df2 = Filtered_df_New[Filtered_df_New['Direction'] == 'phi']
angles_y_axis2 = np.degrees(np.arctan(-eta_df2['Slope']))
angles_y_axis22 = np.degrees(np.arctan(phi_df2['Slope']))
weights2 = 1 / eta_df2.groupby('Event Number')['Event Number'].transform('count')
weights22 = 1 / phi_df2.groupby('Event Number')['Event Number'].transform('count')

# Plotting
plt.figure(figsize=(20, 12))
plt.hist(angles_y_axis2, bins=100, weights=weights2, color='pink', edgecolor='green', alpha = 1, label = 'Atlas On')
plt.hist(angles_y_axis1, bins=100, weights=weights1, color='skyblue', edgecolor='black', alpha = 0.5, label = 'Atlas Off')
plt.xlabel('Angles (degrees)')
plt.ylabel('Weighted Occurance')
plt.title('Cross_Chamber_Angular_Distribution_eta (weighted)')
plt.grid(axis='y', alpha=0.75)
plt.legend()
plt.show()

plt.figure(figsize=(20, 12))
plt.hist(angles_y_axis22, bins=100, weights=weights22, color='pink', edgecolor='green', alpha = 1, label = 'Atlas On')
plt.hist(angles_y_axis11, bins=100, weights=weights11, color='skyblue', edgecolor='black', alpha = 0.5, label = 'Atlas Off')
plt.xlabel('Angles (degrees)')
plt.ylabel('Weighted Occurance')
plt.title('Cross_Chamber_Angular_Distribution_phi (weighted)')
plt.grid(axis='y', alpha=0.75)
plt.legend()
plt.show()

In [190]:
has_eta1 = Filtered_df_Old[Filtered_df_Old['Direction'] == 'eta']['Event Number'].unique()
has_phi1 = Filtered_df_Old[Filtered_df_Old['Direction'] == 'phi']['Event Number'].unique()
events_with_both1 = np.intersect1d(has_eta1, has_phi1)

filtered_df1 = Filtered_df_Old

eta_df1 = Filtered_df_Old[Filtered_df_Old['Direction'] == 'eta']
phi_df1 = Filtered_df_Old[Filtered_df_Old['Direction'] == 'phi']
angles_y_axis1 = np.degrees(np.arctan(-eta_df1['Slope']))
angles_y_axis11 = np.degrees(np.arctan(-phi_df1['Slope']))
weights1 = 1 / eta_df1.groupby('Event Number')['Event Number'].transform('count')
weights11 = 1 / phi_df1.groupby('Event Number')['Event Number'].transform('count')

has_eta2 = Filtered_df_New[Filtered_df_New['Direction'] == 'eta']['Event Number'].unique()
has_phi2 = Filtered_df_New[Filtered_df_New['Direction'] == 'phi']['Event Number'].unique()
events_with_both2 = np.intersect1d(has_eta2, has_phi2)

filtered_df2 = Filtered_df_New[Filtered_df_New['Event Number'].isin(events_with_both2)]

eta_df2 = Filtered_df_New[Filtered_df_New['Direction'] == 'eta']
phi_df2 = Filtered_df_New[Filtered_df_New['Direction'] == 'phi']
angles_y_axis2 = np.degrees(np.arctan(-eta_df2['Slope']))
angles_y_axis22 = np.degrees(np.arctan(phi_df2['Slope']))
weights2 = 1 / eta_df2.groupby('Event Number')['Event Number'].transform('count')
weights22 = 1 / phi_df2.groupby('Event Number')['Event Number'].transform('count')

# Plotting
plt.figure(figsize=(20, 12))
plt.hist(angles_y_axis2, bins=100, weights=weights2, color='pink', edgecolor='green', alpha = 1, label = 'Atlas On')
plt.hist(angles_y_axis1, bins=100, weights=weights1, color='skyblue', edgecolor='black', alpha = 0.5, label = 'Atlas Off')
plt.xlabel('Angles (degrees)')
plt.ylabel('Weighted Occurance')
plt.title('Cross_Chamber_Angular_Distribution_eta (weighted)')
plt.grid(axis='y', alpha=0.75)
plt.legend()
plt.show()

plt.figure(figsize=(20, 12))
plt.hist(angles_y_axis22, bins=100, weights=weights22, color='pink', edgecolor='green', alpha = 1, label = 'Atlas On')
plt.hist(angles_y_axis11, bins=100, weights=weights11, color='skyblue', edgecolor='black', alpha = 0.5, label = 'Atlas Off')
plt.xlabel('Angles (degrees)')
plt.ylabel('Weighted Occurance')
plt.title('Cross_Chamber_Angular_Distribution_phi (weighted)')
plt.grid(axis='y', alpha=0.75)
plt.legend()
plt.show()

In [150]:
y = np.array([-24* 3.09375, -23 * 3.09375, -24 * 3.09375])
x = np.array([0, 0.5, 122.5])
z = np.polyfit(x, y, 1)
print(z)
predicted = z[0] * x + z[1]
residuals = y - predicted
RSS = np.sum(residuals ** 2)
print(RSS)

[-1.25755884e-02 -7.27031509e+01]
4.80517744748802


In [55]:

def parse_times(row):
    coords = ast.literal_eval(row['Used Coordinates'])
    times = {0: [], 1: [], 2: []}

    for coord in coords:
        rpc_id = int(coord[0][3])  
        time = coord[2]
        if rpc_id in [0, 1, 2]:
            times[0].append(time)
        elif rpc_id == 3:
            times[1].append(time)
        elif rpc_id in [4, 5]:
            times[2].append(time)

    for key in times:
        if times[key]:
            times[key] = sum(times[key]) / len(times[key])
        else:
            times[key] = None  

    return times

def determine_direction(times):
    t1, t2, t3 = times[0], times[1], times[2]

    if t1 is not None and t3 is not None:
        return 'bottom' if t1 < t3 else 'top'
    elif t1 is not None and t2 is not None:
        return 'bottom' if t1 < t2 else 'top'
    elif t2 is not None and t3 is not None:
        return 'bottom' if t2 < t3 else 'top'
    else:
        return 'unknown'  

Filtered_df_New['Parsed Times'] = Filtered_df_New.apply(parse_times, axis=1)
Filtered_df_New['Muon Direction'] = Filtered_df_New['Parsed Times'].apply(determine_direction)
Filtered_df_New.drop(columns=['Parsed Times'], inplace=True)
Filtered_df_Old['Parsed Times'] = Filtered_df_Old.apply(parse_times, axis=1)
Filtered_df_Old['Muon Direction'] = Filtered_df_Old['Parsed Times'].apply(determine_direction)
Filtered_df_Old.drop(columns=['Parsed Times'], inplace=True)



In [56]:
def prepare_data(df, direction):
    subset = df[df['Direction'] == direction]
    angles = np.degrees(np.arctan(subset['Slope']))
    weights = 1 / subset.groupby('Event Number')['Event Number'].transform('count')
    # Separate the weights for top and bottom muons
    top_angles = angles[subset['Muon Direction'] == 'top']
    top_weights = weights[subset['Muon Direction'] == 'top']
    bottom_angles = angles[subset['Muon Direction'] == 'bottom']
    bottom_weights = -weights[subset['Muon Direction'] == 'bottom']  # Note the negative sign for bottom
    return top_angles, top_weights, bottom_angles, bottom_weights

angles_eta_new_top, weights_eta_new_top, angles_eta_new_bottom, weights_eta_new_bottom = prepare_data(Filtered_df_New, 'eta')
angles_eta_old_top, weights_eta_old_top, angles_eta_old_bottom, weights_eta_old_bottom = prepare_data(Filtered_df_Old, 'eta')
angles_phi_new_top, weights_phi_new_top, angles_phi_new_bottom, weights_phi_new_bottom = prepare_data(Filtered_df_New, 'phi')
angles_phi_old_top, weights_phi_old_top, angles_phi_old_bottom, weights_phi_old_bottom = prepare_data(Filtered_df_Old, 'phi')

# Plotting all on the same plot
plt.figure(figsize=(20, 12))
plt.hist(angles_eta_old_top, bins=50, weights=weights_eta_old_top, color='skyblue', alpha=0.75, label='Old (Top)')
plt.hist(angles_eta_new_top, bins=50, weights=weights_eta_new_top, color='pink', alpha=0.75, label='AtlasOn (Top)')
plt.hist(angles_eta_new_bottom, bins=50, weights=weights_eta_new_bottom, color='red', alpha=0.75, label='AtlasOn (Bottom)')
plt.hist(angles_eta_old_bottom, bins=50, weights=weights_eta_old_bottom, color='blue', alpha=0.75, label='Old (Bottom)')
plt.xlabel('Angles (degrees)')
plt.ylabel('Weighted Occurrence (Positive for Top, Negative for Bottom)')
plt.title('Cross_Chamber_Angular_Distribution_eta (weighted)')
plt.grid(axis='y', alpha=0.75)
plt.legend()
plt.show()

plt.figure(figsize=(20, 12))
plt.hist(angles_phi_old_top, bins=50, weights=weights_phi_old_top, color='skyblue', alpha=0.75, label='Old (Top)')
plt.hist(angles_phi_new_top, bins=50, weights=weights_phi_new_top, color='pink', alpha=0.75, label='AtlasOn (Top)')
plt.hist(angles_phi_new_bottom, bins=50, weights=weights_phi_new_bottom, color='red', alpha=0.75, label='AtlasOn (Bottom)')
plt.hist(angles_phi_old_bottom, bins=50, weights=weights_phi_old_bottom, color='blue', alpha=0.75, label='Old (Bottom)')
plt.xlabel('Angles (degrees)')
plt.ylabel('Weighted Occurrence (Positive for Top, Negative for Bottom)')
plt.title('Cross_Chamber_Angular_Distribution_phi (weighted)')
plt.grid(axis='y', alpha=0.75)
plt.legend()
plt.show()

In [123]:
def parse_used_coordinates(coord_input):
    # Check if the input is a string that needs parsing
    if isinstance(coord_input, str):
        return ast.literal_eval(coord_input)
    # If it's already the right format (list or set), return it directly
    return coord_input

# Re-define the aggregate_paths function to properly aggregate data
def aggregate_paths(df):
    # Ensure coordinates are parsed correctly
    df['Parsed Coordinates'] = df['Used Coordinates'].apply(parse_used_coordinates)
    # Aggregate all unique RPCs observed in all paths within each event
    aggregated_data = df.groupby('Event Number').agg({
        'Parsed Coordinates': lambda x: {rpc[0] for sublist in x for rpc in sublist},
        'Muon Direction': 'first'  # Assuming all entries for a single event have the same direction
    })
    return aggregated_data

def analyze_event(row):
    observed_rpcs = row['Parsed Coordinates']
    direction = row['Muon Direction']
    expected_sequence = rpc_sequence if direction == 'bottom' else list(reversed(rpc_sequence))

    false_detection = set()
    decay_detection = set()

    # Get indices of observed RPCs in the expected sequence
    observed_indices = [expected_sequence.index(rpc) for rpc in observed_rpcs if rpc in expected_sequence]
    min_observed_index = min(observed_indices, default=None)
    max_observed_index = max(observed_indices, default=None)

    for i, rpc in enumerate(expected_sequence):
        # Sandwiched False logic
        if rpc not in observed_rpcs:
            if i > 0 and i < len(expected_sequence) - 1:
                if expected_sequence[i - 1] in observed_rpcs and expected_sequence[i + 1] in observed_rpcs:
                    false_detection.add(rpc)

    # Decay detection logic
    if direction == 'top' and min_observed_index is not None:
        # Mark RPCs above the first detected RPC as decay
        for i in range(min_observed_index):
            decay_detection.add(expected_sequence[i])
    elif direction == 'bottom' and max_observed_index is not None:
        # Mark RPCs below the last detected RPC as decay
        for i in range(max_observed_index + 1, len(expected_sequence)):
            decay_detection.add(expected_sequence[i])

    return pd.Series({'False': false_detection, 'Decay': decay_detection})

rpc_sequence = ['rpc0', 'rpc1', 'rpc2', 'rpc3', 'rpc4', 'rpc5']
aggregated_df = aggregate_paths(Filtered_df_New)
results = aggregated_df.apply(analyze_event, axis=1)
aggregated_df = pd.concat([aggregated_df, results], axis=1)
aggregated_df2 = aggregate_paths(Filtered_df_Old)
results2 = aggregated_df2.apply(analyze_event, axis=1)
aggregated_df2 = pd.concat([aggregated_df2, results2], axis=1)

print(aggregated_df[['False', 'Decay']])

                     False         Decay
Event Number                            
1             {rpc4, rpc2}            {}
3                       {}  {rpc4, rpc5}
7                       {}            {}
10                      {}  {rpc4, rpc5}
11                      {}            {}
...                    ...           ...
94190               {rpc3}        {rpc5}
94193                   {}            {}
94198                   {}  {rpc4, rpc5}
94204                   {}            {}
94223                   {}            {}

[34168 rows x 2 columns]


In [124]:
show(aggregated_df2.head(100))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [81]:
show(df.head(10000))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [130]:
def union_sets(series):
    return set().union(*series)

# Group by Event Number and union the sets
grouped_false1 = aggregated_df.groupby('Event Number')['False'].apply(union_sets).explode().reset_index()
grouped_false2 = aggregated_df2.groupby('Event Number')['False'].apply(union_sets).explode().reset_index()
grouped_decay = aggregated_df2.groupby('Event Number')['Decay'].apply(union_sets).explode().reset_index()

# Count occurrences
false_counts1 = grouped_false1['False'].value_counts()
false_counts2 = grouped_false2['False'].value_counts()
decay_counts = grouped_decay['Decay'].value_counts()

# Total number of events (using the index of the grouped DataFrames)
total_events1 = grouped_false1.index.nunique()
total_events2 = grouped_false2.index.nunique()

# Calculate inefficiency (False) and decay rates
false_rates1 = false_counts1 / total_events1
false_rates2 = false_counts2 / total_events2
# decay_rates = decay_counts / total_events

# Plotting
fig, ax = plt.subplots(2, 1, figsize=(10, 12))
false_rates1.plot(kind='bar', ax=ax[0], color='red', title='RPC FAILED Detection Rate Atlas On')
false_rates2.plot(kind='bar', ax=ax[1], color='green', title='RPC FAILED Detection Rate Atlas OFF')

plt.tight_layout()
plt.show()

In [76]:
print(false_rates)

rpc2    4.747776
rpc0    4.613439
rpc1    4.487093
rpc3    3.844708
rpc4    2.241513
Name: False, dtype: float64


In [122]:
show(Filtered_df_New.head(100))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


Now I am going to go Barack all the way to the data, and find more insight on timing

In [11]:
fully_massaged_df = cluster_df[cluster_df['size'] <= 5]

In [57]:
print(fully_massaged_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 815112 entries, 0 to 839726
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   event_number     815112 non-null  int64 
 1   rpc_number       815112 non-null  object
 2   strip_direction  815112 non-null  object
 3   locations        815112 non-null  object
 4   times            815112 non-null  object
 5   start_time       815112 non-null  int64 
 6   end_time         815112 non-null  int64 
 7   size             815112 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 56.0+ MB
None


In [10]:
show(fully_massaged_df.head(1000))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [28]:
def calculate_metric_for_combo(combo, rpc_heights):
    locations = np.array([c['location'] for c in combo])
    heights = np.array([rpc_heights[c['rpc']] for c in combo])
    uncertainties = np.array([c['uncertainty'] for c in combo])  
    weights = 1 / uncertainties**2

    try:
        coeffs= np.polyfit(heights, locations, 1, cov=False, w=weights)
    except np.linalg.LinAlgError:
        coeffs = [np.inf, np.inf]
        # cov = np.array([[np.inf, np.inf], [np.inf, np.inf]])

    slope, intercept = coeffs
    # slope_error, intercept_error = np.sqrt(np.diag(cov))
    slope_error, intercept_error = 0, 0
    predicted = slope * heights + intercept
    residuals = locations - predicted
    RSS = np.sum(residuals ** 2)

    return slope, intercept, slope_error, intercept_error, RSS, combo

def analyze_inter_rpc_hit_with_timing_adjusted(df,n):
    rpc_time_offsets = {
    ('rpc0', 'eta'): (7.94, 12.48),
    ('rpc0', 'phi'): (-2.38, 13.69),
    ('rpc1', 'eta'): (8.36, 12.22),
    ('rpc1', 'phi'): (-3.79, 13.25),
    ('rpc2', 'eta'): (8.84, 12.56),
    ('rpc2', 'phi'): (-4.35, 13.57),
    ('rpc3', 'eta'): (6.86, 12.41),
    ('rpc3', 'phi'): (-4.3, 13.96),
    ('rpc4', 'eta'): (2.7, 12.37),
    ('rpc4', 'phi'): (-7.89, 13.41),
    ('rpc5', 'eta'): (2.82, 13.05),
    ('rpc5', 'phi'): (9.15, 14.14),
}
    paths = []

    adjusted_muon_speed_cm_ns = 28

    rpc_heights = {
        'rpc0': 0, 
        'rpc1': 0.5, 
        'rpc2': 1.0, 
        'rpc3': 61.5, 
        'rpc4': 121.5, 
        'rpc5': 122.0
    }

    for event_number, event_group in tqdm(df.groupby('event_number'), desc="Processing Events"):
        for direction in ['eta', 'phi']:
            direction_group = event_group[event_group['strip_direction'] == direction]
            all_clusters = []

            unique_rpcs = direction_group['rpc_number'].unique()
            if len(unique_rpcs) < n:
                break
            for rpc in unique_rpcs:
                rpc_group = direction_group[direction_group['rpc_number'] == rpc]
                for _, row in rpc_group.iterrows():
                    location_scaling = 3.09375 if direction == 'eta' else 2.8125
                    strip_locations = np.array(row['locations'])
                    non_zero_locations = strip_locations[strip_locations != 0]
                    if non_zero_locations.size > 0:
                        strip_location = non_zero_locations[0] 
                    else:
                        continue 

                    location = strip_location * location_scaling
                    event_time = np.mean(row['times']) - rpc_time_offsets[(rpc, direction)][0]
                    cluster_size_scaled = max(row['size'] * location_scaling, location_scaling) / 2
                    all_clusters.append({
                        'rpc': rpc,
                        'location': location,
                        'event_time': event_time,
                        'uncertainty': cluster_size_scaled,
                        'original_location': strip_location
                    })

            combination_metrics = []
            valid_combinations = [] 
            
            for combo in combinations(all_clusters, n):
                if len({c['rpc'] for c in combo}) != len(combo):
                    continue  # Skip combinations where RPCs are not unique

                time_diffs_are_valid = True
                for i in range(len(combo)):
                    for j in range(i + 1, len(combo)):
                        error_window = rpc_time_offsets[(combo[i]['rpc'], direction)][1] + rpc_time_offsets[(combo[j]['rpc'], direction)][1]
                        height_diff = abs(rpc_heights[combo[i]['rpc']] - rpc_heights[combo[j]['rpc']])
                        time_diff = abs(combo[i]['event_time'] - combo[j]['event_time'])
                        expected_time_diff = height_diff / adjusted_muon_speed_cm_ns
                        uncertainty_margin = 5

                        if not (time_diff <= expected_time_diff + error_window + uncertainty_margin):
                            time_diffs_are_valid = False
                            break
                    if not time_diffs_are_valid:
                        break

                if not time_diffs_are_valid:
                    continue  # Skip to the next combination if time differences are invalid

                # Calculate metrics only for valid combinations
                metric = calculate_metric_for_combo(combo, rpc_heights)
                if metric[4] <= 10:
                    valid_combinations.append(metric)
                    

            for valid_combination in valid_combinations:
                paths.append({
                    'Event Number': event_number,
                    'Direction': direction,
                    'Slope': valid_combination[0],
                    'Intercept': valid_combination[1],
                    'Slope_error': valid_combination[2],
                    'Intercept_error': valid_combination[3],
                    'Used Coordinates': [(c['rpc'], c['original_location'], c['event_time']) for c in valid_combination[-1]],
                    'RSS': valid_combination[4],
                })

    path_df = pd.DataFrame(paths)
    return path_df

In [14]:
df_no_rpc0 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc0']
df_no_rpc1 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc1']
df_no_rpc2 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc2']
df_no_rpc3 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc3']
df_no_rpc4 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc4']
df_no_rpc5 = fully_massaged_df[fully_massaged_df['rpc_number'] != 'rpc5']

In [13]:
show(df_no_rpc0)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [15]:
df_recon3 = analyze_inter_rpc_hit_with_timing_adjusted(fully_massaged_df, 3)
df_recon4 = analyze_inter_rpc_hit_with_timing_adjusted(fully_massaged_df, 4)
df_recon5 = analyze_inter_rpc_hit_with_timing_adjusted(fully_massaged_df, 5)
df_recon6 = analyze_inter_rpc_hit_with_timing_adjusted(fully_massaged_df, 6)

Processing Events: 100%|██████████| 94229/94229 [09:32<00:00, 164.50it/s] 


In [16]:
df_recon3.to_excel('df_recon3_AtlasOn_2304.xlsx')
df_recon4.to_excel('df_recon4_AtlasOn_2304.xlsx')
df_recon5.to_excel('df_recon5_AtlasOn_2304.xlsx')
df_recon6.to_excel('df_recon6_AtlasOn_2304.xlsx')

In [17]:
df_recon6_filtered = df_recon6.groupby(['Event Number', 'Direction'], group_keys=False) \
              .apply(lambda x: x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction']).head(1)

df_recon5_filtered = df_recon5.groupby(['Event Number', 'Direction'], group_keys=False) \
              .apply(lambda x: x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction']).head(1)

df_recon4_filtered = df_recon4.groupby(['Event Number', 'Direction'], group_keys=False) \
              .apply(lambda x: x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction']).head(1)
              
df_recon3_filtered = df_recon3.groupby(['Event Number', 'Direction'], group_keys=False) \
              .apply(lambda x: x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction']).head(1)

In [18]:
def count_event_types(df):
    event_directions = {}
    counts = np.array([0, 0, 0])

    for index, row in df.iterrows():
        event_number = row['Event Number']
        direction = row['Direction']

        if event_number not in event_directions:
            event_directions[event_number] = set()
        
        event_directions[event_number].add(direction)

    for directions in event_directions.values():
        if directions == {'eta'}:
            counts[0] += 1
        elif directions == {'phi'}:
            counts[1] += 1
        elif directions == {'eta', 'phi'}:
            counts[2] += 1

    return counts

# result = count_event_types(df_recon6_filtered)
# print("Counts [Only Eta, Only Phi, Both Eta and Phi]:", result)

In [19]:
result6 = count_event_types(df_recon6_filtered)
result5 = count_event_types(df_recon5_filtered)
result4 = count_event_types(df_recon4_filtered)
result3 = count_event_types(df_recon3_filtered)

In [20]:
efficiency6 = result6[2] / (result6[0] + result6[2] + result6[1])
efficiency5 = result5[2] / (result5[0] + result5[2] + result5[1])
efficiency4 = result4[2] / (result4[0] + result4[2] + result4[1])
efficiency3 = result3[2] / (result3[0] + result3[2] + result3[1])

In [21]:
x_labels = [3, 4, 5, 6]

efficiencies = [efficiency3, efficiency4, efficiency5, efficiency6]

plt.bar(x_labels, efficiencies, color='blue')
plt.title('eta phi Efficiency/number of plates Plot')
plt.xlabel('plate used for reconstruction')
plt.ylabel('Efficiency')

plt.show()

In [29]:
df_recon_no_rpc0_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc0, 5)
df_recon_no_rpc1_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc1, 5)
df_recon_no_rpc2_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc2, 5)
df_recon_no_rpc3_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc3, 5)
df_recon_no_rpc4_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc4, 5)
df_recon_no_rpc5_4 = analyze_inter_rpc_hit_with_timing_adjusted(df_no_rpc5, 5)

Processing Events: 100%|██████████| 94226/94226 [01:21<00:00, 1162.84it/s]


In [44]:
show(df_recon_no_rpc5_4)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [30]:
df_recon_no_rpc0_4.to_excel('df_recon_no_rpc0_5_AtlasOn_2304.xlsx')
df_recon_no_rpc1_4.to_excel('df_recon_no_rpc1_5_AtlasOn_2304.xlsx')
df_recon_no_rpc2_4.to_excel('df_recon_no_rpc2_5_AtlasOn_2304.xlsx')
df_recon_no_rpc3_4.to_excel('df_recon_no_rpc3_5_AtlasOn_2304.xlsx')
df_recon_no_rpc4_4.to_excel('df_recon_no_rpc4_5_AtlasOn_2304.xlsx')
df_recon_no_rpc5_4.to_excel('df_recon_no_rpc5_5_AtlasOn_2304.xlsx')

In [27]:
show(df_recon_no_rpc0_4)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI
C:\Users\Peter\AppData\Local\Temp\ipykernel_53540\1082192393.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  show(df_recon_no_rpc0_4)
C:\Users\Peter\AppData\Local\Temp\ipykernel_53540\1082192393.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  show(df_recon_no_rpc0_4)
C:\Users\Peter\AppData\Local\Temp\ipykernel_53540\1082192393.py:1: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

C:\Users\Peter\AppData\Local\Temp\ipykernel_53540\1082192393.py:1: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future vers

In [ ]:
df_recon_no_rpc0_4 = pd.read_excel('df_recon_no_rpc0_4_AtlasOn.xlsx')
df_recon_no_rpc1_4 = pd.read_excel('df_recon_no_rpc1_4_AtlasOn.xlsx')
df_recon_no_rpc2_4 = pd.read_excel('df_recon_no_rpc2_4_AtlasOn.xlsx')
df_recon_no_rpc3_4 = pd.read_excel('df_recon_no_rpc3_4_AtlasOn.xlsx')
df_recon_no_rpc4_4 = pd.read_excel('df_recon_no_rpc4_4_AtlasOn.xlsx')
df_recon_no_rpc5_4 = pd.read_excel('df_recon_no_rpc5_4_AtlasOn.xlsx')

In [31]:
df_recon_no_rpc0_4_filtered = df_recon_no_rpc0_4.groupby(['Event Number', 'Direction'], group_keys=False) \
              .apply(lambda x: x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction']).head(1)

df_recon_no_rpc1_4_filtered = df_recon_no_rpc1_4.groupby(['Event Number', 'Direction'], group_keys=False) \
              .apply(lambda x: x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction']).head(1)

df_recon_no_rpc2_4_filtered = df_recon_no_rpc2_4.groupby(['Event Number', 'Direction'], group_keys=False) \
              .apply(lambda x: x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction']).head(1)
              
df_recon_no_rpc3_4_filtered = df_recon_no_rpc3_4.groupby(['Event Number', 'Direction'], group_keys=False) \
              .apply(lambda x: x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction']).head(1)

df_recon_no_rpc4_4_filtered = df_recon_no_rpc4_4.groupby(['Event Number', 'Direction'], group_keys=False) \
              .apply(lambda x: x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction']).head(1)
              
df_recon_no_rpc5_4_filtered = df_recon_no_rpc5_4.groupby(['Event Number', 'Direction'], group_keys=False) \
              .apply(lambda x: x.sort_values('RSS').iloc[:-1]) \
              .groupby(['Event Number', 'Direction']).head(1)

In [34]:
df_recon_no_rpc0_4_filtered['expected hits'] = df_recon_no_rpc0_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)
df_recon_no_rpc1_4_filtered['expected hits'] = df_recon_no_rpc1_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)
df_recon_no_rpc2_4_filtered['expected hits'] = df_recon_no_rpc2_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)
df_recon_no_rpc3_4_filtered['expected hits'] = df_recon_no_rpc3_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)
df_recon_no_rpc4_4_filtered['expected hits'] = df_recon_no_rpc4_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)
df_recon_no_rpc5_4_filtered['expected hits'] = df_recon_no_rpc5_4_filtered.apply(lambda row: (row['Slope'] * 0 + row['Intercept']) / 3.09375 if row['Direction'] == 'eta' 
                               else (row['Slope'] * 0 + row['Intercept']) / 2.8125 if row['Direction'] == 'phi' 
                               else None, axis=1)

In [37]:
show(df_recon_no_rpc0_4_filtered.head(100))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [53]:
show(fully_massaged_df.head(100))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [35]:
fully_massaged_df_rpc0 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc0']
fully_massaged_df_rpc1 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc1']
fully_massaged_df_rpc2 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc2']
fully_massaged_df_rpc3 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc3']
fully_massaged_df_rpc4 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc4']
fully_massaged_df_rpc5 = fully_massaged_df[fully_massaged_df['rpc_number'] == 'rpc5']

def calculate_midpoint(locations, direction):
    if direction == 'eta':
        return np.mean([loc[1] for loc in locations])
    elif direction == 'phi':
        return np.mean([loc[0] for loc in locations])
    return None

# Adding a column to rpc0_df2 for the midpoint
fully_massaged_df_rpc0['midpoint'] = fully_massaged_df_rpc0.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)
fully_massaged_df_rpc1['midpoint'] = fully_massaged_df_rpc1.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)
fully_massaged_df_rpc2['midpoint'] = fully_massaged_df_rpc2.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)
fully_massaged_df_rpc3['midpoint'] = fully_massaged_df_rpc3.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)
fully_massaged_df_rpc4['midpoint'] = fully_massaged_df_rpc4.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)
fully_massaged_df_rpc5['midpoint'] = fully_massaged_df_rpc5.apply(lambda row: calculate_midpoint(row['locations'], row['strip_direction']), axis=1)

# Merging dfs to compare
merged_df0 = pd.merge(df_recon_no_rpc0_4_filtered, fully_massaged_df_rpc0, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')
merged_df1 = pd.merge(df_recon_no_rpc1_4_filtered, fully_massaged_df_rpc1, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')
merged_df2 = pd.merge(df_recon_no_rpc2_4_filtered, fully_massaged_df_rpc2, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')
merged_df3 = pd.merge(df_recon_no_rpc3_4_filtered, fully_massaged_df_rpc3, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')
merged_df4 = pd.merge(df_recon_no_rpc4_4_filtered, fully_massaged_df_rpc4, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')
merged_df5 = pd.merge(df_recon_no_rpc5_4_filtered, fully_massaged_df_rpc5, left_on=['Event Number', 'Direction'], right_on=['event_number', 'strip_direction'], how='left')


merged_df0['hit_difference'] = merged_df0['expected hits'] - merged_df0['midpoint']
merged_df1['hit_difference'] = merged_df1['expected hits'] - merged_df1['midpoint']
merged_df2['hit_difference'] = merged_df2['expected hits'] - merged_df2['midpoint']
merged_df3['hit_difference'] = merged_df3['expected hits'] - merged_df3['midpoint']
merged_df4['hit_difference'] = merged_df4['expected hits'] - merged_df4['midpoint']
merged_df5['hit_difference'] = merged_df5['expected hits'] - merged_df5['midpoint']

result_df0 = merged_df0[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]
result_df1 = merged_df1[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]
result_df2 = merged_df2[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]
result_df3 = merged_df3[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]
result_df4 = merged_df4[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]
result_df5 = merged_df5[['Event Number', 'Direction', 'expected hits', 'rpc_number', 'midpoint', 'hit_difference']]


C:\Users\Peter\AppData\Local\Temp\ipykernel_53540\2424636361.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Peter\AppData\Local\Temp\ipykernel_53540\2424636361.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Peter\AppData\Local\Temp\ipykernel_53540\2424636361.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [36]:
eta_df0 = result_df0[result_df0['Direction'] == 'eta']
eta_df1 = result_df1[result_df1['Direction'] == 'eta']
eta_df2 = result_df2[result_df2['Direction'] == 'eta']
eta_df3 = result_df3[result_df3['Direction'] == 'eta']
eta_df4 = result_df4[result_df4['Direction'] == 'eta']
eta_df5 = result_df5[result_df5['Direction'] == 'eta']

phi_df0 = result_df0[result_df0['Direction'] == 'phi']
phi_df1 = result_df1[result_df1['Direction'] == 'phi']
phi_df2 = result_df2[result_df2['Direction'] == 'phi']
phi_df3 = result_df3[result_df3['Direction'] == 'phi']
phi_df4 = result_df4[result_df4['Direction'] == 'phi']
phi_df5 = result_df5[result_df5['Direction'] == 'phi']

In [37]:
def get_min_index(group):
    if group['hit_difference'].isna().all():
        return group.index.tolist()  # Return all indices if all are NaN
    return [group['hit_difference'].idxmin()]


In [38]:
phi_df0 = phi_df0.loc[phi_df0.groupby('Event Number').apply(get_min_index).explode().astype(int)]
phi_df1 = phi_df1.loc[phi_df1.groupby('Event Number').apply(get_min_index).explode().astype(int)]
phi_df2 = phi_df2.loc[phi_df2.groupby('Event Number').apply(get_min_index).explode().astype(int)]
phi_df3 = phi_df3.loc[phi_df3.groupby('Event Number').apply(get_min_index).explode().astype(int)]
phi_df4 = phi_df4.loc[phi_df4.groupby('Event Number').apply(get_min_index).explode().astype(int)]
phi_df5 = phi_df5.loc[phi_df5.groupby('Event Number').apply(get_min_index).explode().astype(int)]

eta_df0 = eta_df0.loc[eta_df0.groupby('Event Number').apply(get_min_index).explode().astype(int)]
eta_df1 = eta_df1.loc[eta_df1.groupby('Event Number').apply(get_min_index).explode().astype(int)]
eta_df2 = eta_df2.loc[eta_df2.groupby('Event Number').apply(get_min_index).explode().astype(int)]
eta_df3 = eta_df3.loc[eta_df3.groupby('Event Number').apply(get_min_index).explode().astype(int)]
eta_df4 = eta_df4.loc[eta_df4.groupby('Event Number').apply(get_min_index).explode().astype(int)]
eta_df5 = eta_df5.loc[eta_df5.groupby('Event Number').apply(get_min_index).explode().astype(int)]



In [111]:
print(eta_df)

      Event Number Direction  expected hits rpc_number  midpoint  \
0               31       eta     -13.943812       rpc0     -14.0   
1               51       eta     -24.960487       rpc0     -24.0   
2              129       eta     -23.982698       rpc0     -24.0   
3              157       eta      -6.959865       rpc0      -7.0   
5              161       eta      -6.895650        NaN       NaN   
...            ...       ...            ...        ...       ...   
2364         92107       eta     -14.028003       rpc0     -14.0   
2365         92161       eta      -6.746919       rpc0     -31.0   
2366         92161       eta      -6.746919       rpc0      -5.0   
2367         92161       eta      -6.746919       rpc0     -19.0   
2370         92238       eta     -19.055707       rpc0     -19.0   

      hit_difference  
0           0.056188  
1          -0.960487  
2           0.017302  
3           0.040135  
5                NaN  
...              ...  
2364       -0.028003  

In [39]:
thresholds = range(33)  

eta_df = [eta_df0, eta_df1, eta_df2, eta_df3, eta_df4, eta_df5]

efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

for i in range(6):
    for threshold in thresholds:
        valid_hits = eta_df[i].dropna(subset=['hit_difference'])

        count_valid = valid_hits[(valid_hits['hit_difference'].abs() <= threshold) & (~valid_hits['hit_difference'].isna())].shape[0]

        total_valid = eta_df[i].shape[0]

        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)

plt.figure(figsize=(20, 12))
for i in range(6):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for ETA Direction RPC0 any 4 AtlasOn')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()

In [40]:
thresholds = range(63)  

phi_df = [phi_df0, phi_df1, phi_df2, phi_df3, phi_df4, phi_df5]

efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

for i in range(6):
    for threshold in thresholds:
        valid_hits = phi_df[i].dropna(subset=['hit_difference'])

        count_valid = valid_hits[(valid_hits['hit_difference'].abs() <= threshold) & (~valid_hits['hit_difference'].isna())].shape[0]

        total_valid = phi_df[i].shape[0]

        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)

plt.figure(figsize=(20, 12))
for i in range(6):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for PHI Direction RPC0 any 4 Atlas On')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()

In [58]:
print(phi_df1.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129 entries, 0 to 339
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Event Number    129 non-null    int64  
 1   Direction       129 non-null    object 
 2   expected hits   129 non-null    float64
 3   rpc_number      124 non-null    object 
 4   midpoint        124 non-null    float64
 5   hit_difference  124 non-null    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 7.1+ KB
None


In [68]:
show(phi_df1)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [76]:
show(result_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [41]:
combined_df = [[],[],[],[],[],[]]

efficiencies = [[],[],[],[],[],[]]

label_rpc = ['missing rpc0', 'missing rpc1', 'missing rpc2', 'missing rpc3', 'missing rpc4', 'missing rpc5']

thresholds = range(70)  

for i in range(6):
    combined_df[i] = pd.merge(eta_df[i], phi_df[i], on='Event Number', how='inner')
    
    
for i in range(6):
    for threshold in thresholds:
        valid_hits = combined_df[i].dropna(subset=['hit_difference_x', 'hit_difference_y'], how='any')

        # Calculate the number of valid hits within the threshold
        count_valid = valid_hits[((valid_hits['hit_difference_x'].abs()) ** 2 + (valid_hits['hit_difference_y'].abs()) ** 2 <= threshold ** 2)].shape[0]

        # Total number of rows after dropping NaN values
        total_valid = combined_df[i].shape[0]

        # Calculate efficiency
        
        efficiency = count_valid / total_valid if total_valid > 0 else 0
        efficiencies[i].append(efficiency)


plt.figure(figsize=(20, 12))
for i in range(6):
    plt.plot(thresholds, efficiencies[i], marker='o', label = label_rpc[i])
plt.title('Efficiency vs Hit Difference Threshold for Eta Phi Combined any 4 Atlas On')
plt.xlabel('Hit Difference Threshold in Number of Strips')
plt.ylabel('Efficiency')
plt.legend()
plt.grid(True)
plt.show()
    

In [56]:
show(fully_massaged_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


Now for each individual TDC

In [10]:
def calculate_metric_for_combo(combo, rpc_heights):
    locations = np.array([c['location'] for c in combo])
    heights = np.array([rpc_heights[c['rpc']] for c in combo])
    uncertainties = np.array([c['uncertainty'] for c in combo])  
    weights = 1 / uncertainties**2

    try:
        coeffs= np.polyfit(heights, locations, 1, cov=False, w=weights)
    except np.linalg.LinAlgError:
        coeffs = [np.inf, np.inf]
        # cov = np.array([[np.inf, np.inf], [np.inf, np.inf]])

    slope, intercept = coeffs
    # slope_error, intercept_error = np.sqrt(np.diag(cov))
    slope_error, intercept_error = 0, 0
    predicted = slope * heights + intercept
    residuals = locations - predicted
    RSS = np.sum(residuals ** 2)

    return slope, intercept, slope_error, intercept_error, RSS, combo

def analyze_inter_rpc_hit_with_timing_adjusted(df, n):
    rpc_time_offsets = {
    ('rpc0', 'eta'): (7.94, 12.48),
    ('rpc0', 'phi'): (-2.38, 13.69),
    ('rpc1', 'eta'): (8.36, 12.22),
    ('rpc1', 'phi'): (-3.79, 13.25),
    ('rpc2', 'eta'): (8.84, 12.56),
    ('rpc2', 'phi'): (-4.35, 13.57),
    ('rpc3', 'eta'): (6.86, 12.41),
    ('rpc3', 'phi'): (-4.3, 13.96),
    ('rpc4', 'eta'): (2.7, 12.37),
    ('rpc4', 'phi'): (-7.89, 13.41),
    ('rpc5', 'eta'): (2.82, 13.05),
    ('rpc5', 'phi'): (9.15, 14.14),
}
    paths = []

    adjusted_muon_speed_cm_ns = 30

    rpc_heights = {
        'rpc0': 0, 
        'rpc1': 0.5, 
        'rpc2': 1.0, 
        'rpc3': 61.5, 
        'rpc4': 121.5, 
        'rpc5': 122.0
    }

    for event_number, event_group in tqdm(df.groupby('event_number'), desc="Processing Events"):
        for direction in ['eta', 'phi']:
            direction_group = event_group[event_group['strip_direction'] == direction]
            all_clusters = []

            unique_rpcs = direction_group['rpc_number'].unique()
            if len(unique_rpcs) < n:
                break
            for rpc in unique_rpcs:
                rpc_group = direction_group[direction_group['rpc_number'] == rpc]
                for _, row in rpc_group.iterrows():
                    location_scaling = 3.09375 if direction == 'eta' else 2.8125
                    strip_locations = np.array(row['locations'])
                    non_zero_locations = strip_locations[strip_locations != 0]
                    if non_zero_locations.size > 0:
                        strip_location = non_zero_locations[0] 
                    else:
                        continue 

                    location = strip_location * location_scaling
                    event_time = np.mean(row['times']) - rpc_time_offsets[(rpc, direction)][0]
                    cluster_size_scaled = max(row['size'] * location_scaling, location_scaling) / 2
                    all_clusters.append({
                        'rpc': rpc,
                        'location': location,
                        'event_time': event_time,
                        'uncertainty': cluster_size_scaled,
                        'original_location': strip_location
                    })

            combination_metrics = []
            valid_combinations = [] 
            
            for combo in combinations(all_clusters, n):
                if len({c['rpc'] for c in combo}) != len(combo):
                    continue  # Skip combinations where RPCs are not unique

                time_diffs_are_valid = True
                for i in range(len(combo)):
                    for j in range(i + 1, len(combo)):
                        error_window = rpc_time_offsets[(combo[i]['rpc'], direction)][1] + rpc_time_offsets[(combo[j]['rpc'], direction)][1]
                        height_diff = abs(rpc_heights[combo[i]['rpc']] - rpc_heights[combo[j]['rpc']])
                        time_diff = abs(combo[i]['event_time'] - combo[j]['event_time'])
                        expected_time_diff = height_diff / adjusted_muon_speed_cm_ns
                        uncertainty_margin = 5

                        if not (time_diff <= expected_time_diff + error_window + uncertainty_margin):
                            time_diffs_are_valid = False
                            break
                    if not time_diffs_are_valid:
                        break

                if not time_diffs_are_valid:
                    continue  # Skip to the next combination if time differences are invalid

                # Calculate metrics only for valid combinations
                metric = calculate_metric_for_combo(combo, rpc_heights)
                if metric[4] <= 10:
                    valid_combinations.append(metric)
                    

            for valid_combination in valid_combinations:
                paths.append({
                    'Event Number': event_number,
                    'Direction': direction,
                    'Slope': valid_combination[0],
                    'Intercept': valid_combination[1],
                    'Slope_error': valid_combination[2],
                    'Intercept_error': valid_combination[3],
                    'Used Coordinates': [(c['rpc'], c['original_location'], c['event_time']) for c in valid_combination[-1]],
                    'RSS': valid_combination[4],
                })

    path_df = pd.DataFrame(paths)
    return path_df

Now do the timing adjustment using the bottom triplet

In [30]:
def calculate_metric_for_combo(combo, rpc_heights):
    locations = np.array([c['location'] for c in combo])
    heights = np.array([rpc_heights[c['rpc']] for c in combo])
    uncertainties = np.array([c['uncertainty'] for c in combo])  
    weights = 1 / uncertainties**2

    try:
        coeffs= np.polyfit(heights, locations, 1, cov=False, w=weights)
    except np.linalg.LinAlgError:
        coeffs = [np.inf, np.inf]
        # cov = np.array([[np.inf, np.inf], [np.inf, np.inf]])

    slope, intercept = coeffs
    # slope_error, intercept_error = np.sqrt(np.diag(cov))
    slope_error, intercept_error = 0, 0
    predicted = slope * heights + intercept
    residuals = locations - predicted
    RSS = np.sum(residuals ** 2)

    return slope, intercept, slope_error, intercept_error, RSS, combo

def analyze_inter_rpc_hit_with_timing_adjusted(df,n):

    rpc_time_offsets = {
    ('rpc0', 'eta'): (0, 5),
    ('rpc0', 'phi'): (0, 5),
    ('rpc1', 'eta'): (0, 5),
    ('rpc1', 'phi'): (0, 5),
    ('rpc2', 'eta'): (0, 5),
    ('rpc2', 'phi'): (0, 5),
    ('rpc3', 'eta'): (0, 5),
    ('rpc3', 'phi'): (0, 5),
    ('rpc4', 'eta'): (0, 5),
    ('rpc4', 'phi'): (0, 5),
    ('rpc5', 'eta'): (0, 5),
    ('rpc5', 'phi'): (0, 5),
}
    paths = []

    adjusted_muon_speed_cm_ns = 30

    rpc_heights = {
        'rpc0': 0, 
        'rpc1': 0.5, 
        'rpc2': 1.0, 
        'rpc3': 61.5, 
        'rpc4': 121.5, 
        'rpc5': 122.0
    }

    for event_number, event_group in tqdm(df.groupby('event_number'), desc="Processing Events"):
        for direction in ['eta', 'phi']:
            direction_group = event_group[event_group['strip_direction'] == direction]
            all_clusters = []

            unique_rpcs = direction_group['rpc_number'].unique()
            if len(unique_rpcs) < 5:
                break
            for rpc in unique_rpcs:
                rpc_group = direction_group[direction_group['rpc_number'] == rpc]
                for _, row in rpc_group.iterrows():
                    location_scaling = 3.09375 if direction == 'eta' else 2.8125
                    strip_locations = np.array(row['locations'])
                    non_zero_locations = strip_locations[strip_locations != 0]
                    if non_zero_locations.size > 0:
                        strip_location = non_zero_locations[0] 
                    else:
                        continue 

                    location = strip_location * location_scaling
                    event_time = np.mean(row['times']) - rpc_time_offsets[(rpc, direction)][0]
                    cluster_size_scaled = max(row['size'] * location_scaling, location_scaling) / 2
                    all_clusters.append({
                        'rpc': rpc,
                        'location': location,
                        'event_time': event_time,
                        'uncertainty': cluster_size_scaled,
                        'original_location': strip_location
                    })

            combination_metrics = []
            valid_combinations = [] 
            
            for combo in combinations(all_clusters, n):
                if len({c['rpc'] for c in combo}) != len(combo):
                    continue  # Skip combinations where RPCs are not unique
                used_rpcs_set = {c['rpc'] for c in combo}
  

                time_diffs_are_valid = True
                for i in range(len(combo)):
                    for j in range(i + 1, len(combo)):
                        error_window = rpc_time_offsets[(combo[i]['rpc'], direction)][1] + rpc_time_offsets[(combo[j]['rpc'], direction)][1]
                        height_diff = abs(rpc_heights[combo[i]['rpc']] - rpc_heights[combo[j]['rpc']])
                        time_diff = abs(combo[i]['event_time'] - combo[j]['event_time'])
                        expected_time_diff = height_diff / adjusted_muon_speed_cm_ns
                        uncertainty_margin = 5

                        if not (time_diff <= expected_time_diff + error_window + uncertainty_margin):
                            time_diffs_are_valid = False
                            break
                    if not time_diffs_are_valid:
                        break

                if not time_diffs_are_valid:
                    continue  # Skip to the next combination if time differences are invalid
                
                if used_rpcs_set.issubset({'rpc0', 'rpc1', 'rpc2'}):
                    metric = calculate_metric_for_combo(combo, rpc_heights)
                    if metric[4] <= 10:
                        valid_combinations.append(metric)
                    

            for valid_combination in valid_combinations:
                paths.append({
                    'Event Number': event_number,
                    'Direction': direction,
                    'Slope': valid_combination[0],
                    'Intercept': valid_combination[1],
                    'Slope_error': valid_combination[2],
                    'Intercept_error': valid_combination[3],
                    'Used Coordinates': [(c['rpc'], c['original_location'], c['event_time']) for c in valid_combination[-1]],
                    'RSS': valid_combination[4],
                })

    path_df = pd.DataFrame(paths)
    return path_df

In [31]:
df_Triplet_timing = analyze_inter_rpc_hit_with_timing_adjusted(fully_massaged_df, 3)

Processing Events: 100%|██████████| 94226/94226 [00:49<00:00, 1892.94it/s]


In [32]:
show(df_Triplet_timing)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI
